In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install import_ipynb 
!conda install -c conda-forge import_ipynb

In [ ]:
import sys

sys.path.append('../')

from game_runner import NegotitaionGame
from eval.game_evaluator import GameEvaluator
import agents.simple_agent as simple_agent
import agents.llm_agent as llm_agent
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from dataclasses import dataclass, field
from math import prod, sqrt
sys.path.append('../caif_negotiation/')

# Import the notebook
#import import_ipynb
#%run '../test_game_eval.ipynb'  
import torch
from utils.offer import Offer

from prompts.make_prompt import make_prompt
from prompts.make_prompt_bargain import make_prompt_bargain
from metrics.visualizations import (
    plot_discounted_values,
    plot_offer_evolution,
    plot_negotiation_gap,
    plot_fairness
)
import concurrent.futures


pathology_results = pd.DataFrame()  
import itertools
envy_results_history = {}
from eval.metrics import *
from utils.helpers import *
from utils.negotiation_game import *



In [4]:
import time
import pandas as pd
import torch
import numpy as np
from math import sqrt, prod

# ------------------------------------------------------------------------
# Configuration
# ------------------------------------------------------------------------
# prompt_style = 'llama_3.3_70b_maximize_value_outside_offer_cot_1_test'
prompt_style = '4o'
llm_type = 'openai'
date = '1_28_2025'
max_rounds = 3
games = 100
circles = [6]



## Run Games

In [ ]:
'''
for circle in circles:
    print(f"Running game for circle {circle}")
    run_game(circle, games, max_rounds, date, prompt_style, llm_type)
'''

with concurrent.futures.ProcessPoolExecutor() as executor:
    future_to_circle = {
        executor.submit(run_game, circle, games, max_rounds, date, prompt_style, llm_type): circle
        for circle in circles
    }
    for future in concurrent.futures.as_completed(future_to_circle):
        circle_val = future_to_circle[future]
        try:
            future.result()
            print(f"[INFO] circle={circle_val} run finished successfully.")
        except Exception as exc:
            print(f"[ERROR] circle={circle_val} generated an exception: {exc}")

## Pathology Computation

In [ ]:
import json
import re
import math
import matplotlib.pyplot as plt
import numpy as np

DISCOUNT_RATE = 0.9

def get_discount_factor(round_index, player_id):
    exponent = (round_index - 1) if player_id == 1 else round_index
    return DISCOUNT_RATE ** exponent

def parse_valuations_from_prompt(prompt_text):
    player_id = None
    if "You are Player 1" in prompt_text:
        player_id = 1
    elif "You are Player 2" in prompt_text:
        player_id = 2
    else:
        raise ValueError("Could not determine player (Player 1 or Player 2) from prompt.")

    item_qty_pattern = re.compile(
        r"There\s+are\s+(\d+)\s+units\s+of\s+item\s+1,\s+(\d+)\s+units\s+of\s+item\s+2,\s+(\d+)\s+units\s+of\s+item\s+3,\s+(\d+)\s+units\s+of\s+item\s+4,\s+(\d+)\s+units\s+of\s+item\s+5"
    )
    item_qty_match = item_qty_pattern.search(prompt_text.replace("\n", " "))
    if not item_qty_match:
        raise ValueError("Could not parse item quantities from prompt.")
    items = list(map(int, item_qty_match.groups()))

    val_pattern = re.compile(
        r"Your\s+private\s+values\s+are\s+(\d+)\s+for\s+item\s+1,\s+(\d+)\s+for\s+item\s+2,\s+(\d+)\s+for\s+item\s+3,\s+(\d+)\s+for\s+item\s+4,\s+(\d+)\s+for\s+item\s+5"
    )
    val_match = val_pattern.search(prompt_text.replace("\n", " "))
    if not val_match:
        raise ValueError("Could not parse valuations from prompt.")
    valuations = list(map(int, val_match.groups()))

    outoffer_pattern = re.compile(r"Your\s+outside\s+offer\s+value\s+is\s+(\d+)")
    outoffer_match = outoffer_pattern.search(prompt_text.replace("\n", " "))
    if not outoffer_match:
        raise ValueError("Could not parse outside offer from prompt.")
    outside_offer = int(outoffer_match.group(1))

    return {
        "player_id": player_id,
        "valuations": valuations,
        "items": items,
        "outside_offer": outside_offer
    }

def parse_offer_from_response(response_text):
    pattern = re.compile(r"\"offer\"\s*:\s*\[\s*(\d+)\s*,\s*(\d+)\s*,\s*(\d+)\s*,\s*(\d+)\s*,\s*(\d+)\s*\]")
    match = pattern.search(response_text)
    return list(map(int, match.groups())) if match else None

def compute_allocation_value(valuations, allocation):
    #check type in valuations & allocations
    if valuations is None or allocation is None:
        print("HERE")
    elif any(x is None for x in valuations) or any(x is None for x in allocation):
        print("HERE2")
    return sum(v * a for v, a in zip(valuations, allocation))

def detect_mistakes(current_move_info, previous_move_info_player):
    mistakes_found = []
    action = current_move_info["action"]
    my_val_cur = current_move_info["my_value_current_offer"]
    my_val_prev = current_move_info["my_value_previous_own_offer"]
    out_offer = current_move_info["my_outside_offer"]
    allocation_to_opp = current_move_info["allocation_offered_to_opponent"]
    allocation_i_keep = current_move_info["allocation_i_keep"]
    is_game_ending = current_move_info["is_game_ending"]

    if action == "COUNTEROFFER" and not is_game_ending:
        if my_val_prev is not None and my_val_prev > my_val_cur:
            mistakes_found.append(1)
        if my_val_cur < out_offer:
            mistakes_found.append(2)
        if allocation_to_opp is not None and allocation_i_keep is not None:
            sum_offered = sum(allocation_to_opp)
            sum_kept = sum(allocation_i_keep)
            total_items = sum_offered + sum_kept
            if total_items > 0:
                if sum_offered == 0 or sum_offered == total_items:
                    mistakes_found.append(3)

    if action == "ACCEPT" and is_game_ending:
        if my_val_cur < out_offer:
            mistakes_found.append(4)

    if action == "WALK" and is_game_ending:
        if my_val_cur > out_offer:
            mistakes_found.append(5)

    return mistakes_found

def analyze_circle6(json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    all_game_data = data["all_game_data"]
    num_games = len(all_game_data)
    num_moves = 0
    moves_by_round = {1: 0, 2: 0, 3: 0}
    moves_by_player = {1: 0, 2: 0}
    mistake_counts = {k: 0 for k in range(1, 6)}
    mistake_counts_by_player = {
        1: {k: 0 for k in range(1, 6)},
        2: {k: 0 for k in range(1, 6)}
    }
    mistake_counts_by_round = {
        1: {k: 0 for k in range(1, 6)},
        2: {k: 0 for k in range(1, 6)},
        3: {k: 0 for k in range(1, 6)},
    }

    for game in all_game_data:
        round_data = game["round_data"]
        last_move_info = {1: None, 2: None}
        total_rounds = 3
        current_round = 1
        i = 0

        # We iterate through at most total_rounds, each round can have up to two moves
        while i < len(round_data) and current_round <= total_rounds:
            # ---------------------------------------
            # Process the first move of this round
            # ---------------------------------------
            first_move = round_data[i]
            i += 1

            try:
                prompt = first_move["prompt"]
                response = first_move["response"]
                action = first_move["action"]
            except KeyError:
                # If something is off, skip this round
                current_round += 1
                continue

            try:
                parsed_info = parse_valuations_from_prompt(prompt)
            except ValueError:
                current_round += 1
                continue

            player_id = parsed_info["player_id"]
            valuations = parsed_info["valuations"]
            items = parsed_info["items"]
            outside_offer = parsed_info["outside_offer"]
            discount_factor = get_discount_factor(current_round, player_id)
            discounted_outside_offer = outside_offer * discount_factor
            is_game_ending = (action in ["ACCEPT", "WALK", "INVALID WALK"])
            offer_allocation = parse_offer_from_response(response) if action == "COUNTEROFFER" else None
            
            if action == "COUNTEROFFER":
                if offer_allocation is not None:
                    allocation_offered_to_opponent = offer_allocation
                    allocation_i_keep = [ti - oa for ti, oa in zip(items, offer_allocation)]
                    current_value_for_self = compute_allocation_value(valuations, allocation_i_keep) * discount_factor
                else:
                    allocation_offered_to_opponent = None
                    allocation_i_keep = None
                    current_value_for_self = 0.0
            elif action == "ACCEPT":
                opp_id = 1 if player_id == 2 else 2
                opp_info = last_move_info[opp_id]
                if opp_info and opp_info["action"] == "COUNTEROFFER":
                    our_alloc = opp_info["allocation_offered_to_opponent"]
                    current_value_for_self = compute_allocation_value(valuations, our_alloc) * discount_factor
                    allocation_offered_to_opponent = None
                    allocation_i_keep = our_alloc
                else:
                    current_value_for_self = 0.0
                    allocation_offered_to_opponent = None
                    allocation_i_keep = None
            elif action in ["WALK", "INVALID WALK"]:
                opp_id = 1 if player_id == 2 else 2
                opp_info = last_move_info[opp_id]
                if opp_info and opp_info["action"] == "COUNTEROFFER":
                    our_alloc = opp_info["allocation_offered_to_opponent"]
                    current_value_for_self = compute_allocation_value(valuations, our_alloc) * discount_factor
                    allocation_offered_to_opponent = None
                    allocation_i_keep = our_alloc
                else:
                    current_value_for_self = 0.0
                    allocation_offered_to_opponent = None
                    allocation_i_keep = None
            else:
                # Action is something we don't track, do not increment round multiple times
                continue

            my_value_previous_own_offer = None
            if last_move_info[player_id] and last_move_info[player_id]["action"] == "COUNTEROFFER":
                my_value_previous_own_offer = last_move_info[player_id]["my_value_current_offer"]

            current_move_info_struct = {
                "player_id": player_id,
                "action": action,
                "my_value_current_offer": current_value_for_self,
                "my_outside_offer": discounted_outside_offer,
                "my_value_previous_own_offer": my_value_previous_own_offer,
                "allocation_offered_to_opponent": allocation_offered_to_opponent,
                "allocation_i_keep": allocation_i_keep,
                "is_game_ending": is_game_ending,
            }

            mistakes_triggered = detect_mistakes(
                current_move_info_struct,
                last_move_info.get(player_id)
            )
            for mk in mistakes_triggered:
                mistake_counts[mk] += 1
                mistake_counts_by_player[player_id][mk] += 1
                mistake_counts_by_round[current_round][mk] += 1

            if action in ["COUNTEROFFER", "ACCEPT", "WALK", "INVALID WALK"]:
                num_moves += 1
                moves_by_round[current_round] += 1
                moves_by_player[player_id] += 1

            if action == "COUNTEROFFER":
                last_move_info[player_id] = {
                    "action": action,
                    "my_value_current_offer": current_value_for_self,
                    "allocation_offered_to_opponent": allocation_offered_to_opponent,
                    "allocation_i_keep": allocation_i_keep
                }
            if is_game_ending:
                current_round += 1
                break

            # ---------------------------------------
            # Process the second move of this round (if any)
            # ---------------------------------------
            if i < len(round_data):
                second_move = round_data[i]
                i += 1

                try:
                    prompt2 = second_move["prompt"]
                    response2 = second_move["response"]
                    
                    action2 = second_move["action"]
                except KeyError:
                    current_round += 1
                    continue

                try:
                    parsed_info2 = parse_valuations_from_prompt(prompt2)
                except ValueError:
                    current_round += 1
                    continue

                player_id2 = parsed_info2["player_id"]
                valuations2 = parsed_info2["valuations"]
                items2 = parsed_info2["items"]
                outside_offer2 = parsed_info2["outside_offer"]
                discount_factor2 = get_discount_factor(current_round, player_id2)
                discounted_outside_offer2 = outside_offer2 * discount_factor2
                is_game_ending2 = (action2 in ["ACCEPT", "WALK", "INVALID WALK"])
                offer_allocation2 = parse_offer_from_response(response2) if action2 == "COUNTEROFFER" else None

                if action2 == "COUNTEROFFER":
                    if offer_allocation2 is not None:
                        allocation_offered_to_opponent2 = offer_allocation2
                        allocation_i_keep2 = [ti - oa for ti, oa in zip(items2, offer_allocation2)]
                        current_value_for_self2 = compute_allocation_value(valuations2, allocation_i_keep2) * discount_factor2
                    else:
                        allocation_offered_to_opponent2 = None
                        allocation_i_keep2 = None
                        current_value_for_self2 = 0.0
                elif action2 == "ACCEPT":
                    opp_id2 = 1 if player_id2 == 2 else 2
                    opp_info2 = last_move_info[opp_id2]
                    if opp_info2 and opp_info2["action"] == "COUNTEROFFER":
                        our_alloc2 = opp_info2["allocation_offered_to_opponent"]
                        
                        current_value_for_self2 = compute_allocation_value(valuations2, our_alloc2) * discount_factor2
                        allocation_offered_to_opponent2 = None
                        allocation_i_keep2 = our_alloc2
                    else:
                        current_value_for_self2 = 0.0
                        allocation_offered_to_opponent2 = None
                        allocation_i_keep2 = None 
                elif action2 in ["WALK", "INVALID WALK"]:
                    opp_id2 = 1 if player_id2 == 2 else 2
                    opp_info2 = last_move_info[opp_id2]
                    if opp_info2 and opp_info2["action"] == "COUNTEROFFER":
                        our_alloc2 = opp_info2["allocation_offered_to_opponent"]
                        #if our_alloc2 is None or valuations2 is None:
                           #print(prompt)
                            #print(response)
                            #print(action)
                        
                        current_value_for_self2 = compute_allocation_value(valuations2, our_alloc2) * discount_factor2
                        allocation_offered_to_opponent2 = None
                        allocation_i_keep2 = our_alloc2
                    else:
                        current_value_for_self2 = 0.0
                        allocation_offered_to_opponent2 = None
                        allocation_i_keep2 = None
                else:
                    # Unknown action, do not truncate the round count
                    current_round += 1
                    continue

                my_value_previous_own_offer2 = None
                if last_move_info[player_id2] and last_move_info[player_id2]["action"] == "COUNTEROFFER":
                    my_value_previous_own_offer2 = last_move_info[player_id2]["my_value_current_offer"]

                current_move_info_struct2 = {
                    "player_id": player_id2,
                    "action": action2,
                    "my_value_current_offer": current_value_for_self2,
                    "my_outside_offer": discounted_outside_offer2,
                    "my_value_previous_own_offer": my_value_previous_own_offer2,
                    "allocation_offered_to_opponent": allocation_offered_to_opponent2,
                    "allocation_i_keep": allocation_i_keep2,
                    "is_game_ending": is_game_ending2,
                }

                mistakes_triggered2 = detect_mistakes(
                    current_move_info_struct2,
                    last_move_info.get(player_id2)
                )
                for mk2 in mistakes_triggered2:
                    mistake_counts[mk2] += 1
                    mistake_counts_by_player[player_id2][mk2] += 1
                    mistake_counts_by_round[current_round][mk2] += 1

                if action2 in ["COUNTEROFFER", "ACCEPT", "WALK", "INVALID WALK"]:
                    num_moves += 1
                    moves_by_round[current_round] += 1
                    moves_by_player[player_id2] += 1

                if action2 == "COUNTEROFFER":
                    last_move_info[player_id2] = {
                        "action": action2,
                        "my_value_current_offer": current_value_for_self2,
                        "allocation_offered_to_opponent": allocation_offered_to_opponent2,
                        "allocation_i_keep": allocation_i_keep2
                    }
                if is_game_ending2:
                    current_round += 1
                    break

            # End of this round if the second move didn't end the game
            else:
                # If there's no second move, we still consider the round done
                pass

            # Move to the next round
            current_round += 1

    def ratio_str(numerator, denominator):
        if denominator == 0:
            return "0/0 (0.0%)"
        perc = (numerator / denominator) * 100.0
        return f"{numerator}/{denominator} ({perc:.1f}%)"

    def print_mistake_results(mk):
        if mk in (4, 5):
            overall_denom = num_games
        else:
            overall_denom = num_moves
        print(f"=== Mistake {mk} ===\n")
        print(f"Overall: {ratio_str(mistake_counts[mk], overall_denom)}")
        for p in [1, 2]:
            if mk in (4, 5):
                denom = num_games
            else:
                denom = moves_by_player[p]
            print(f"Player {p}: {ratio_str(mistake_counts_by_player[p][mk], denom)}")
        for r in [1, 2, 3]:
            if mk in (4, 5):
                denom = num_games
            else:
                denom = moves_by_round[r]
            mk_count = mistake_counts_by_round[r].get(mk, 0)
            print(f"Round {r}: {ratio_str(mk_count, denom)}")
        print()

    for mk in [1, 2, 3, 4, 5]:
        print_mistake_results(mk)
    return mistake_counts

def plot_mistakes_radar_multiple_circles(circles_mistake_counts, model_name):
    """
    Plots a single radar chart with multiple "circles" (series),
    each circle represented by a distinct color.

    circles_mistake_counts: dict where:
       - key: circle name (str), e.g. "circle_0"
       - value: dict of {mistake_id: count}, e.g., {1:10, 2:4, 3:5, 4:3, 5:2}
    """

    # For simplicity, we'll assume mistakes are always 1..5
    labels = [1, 2, 3, 4, 5]
    n_labels = len(labels)

    # Create angles for the radar plot
    angles = np.linspace(0, 2 * np.pi, n_labels, endpoint=False).tolist()
    # Close the circle
    angles += [angles[0]]

    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={"polar": True})

    # Use a color map to assign a different color to each circle
    colors = plt.cm.tab10(np.linspace(0, 1, len(circles_mistake_counts)))

    for (circle_name, mistake_counts), color in zip(circles_mistake_counts.items(), colors):
        # Extract values in the sorted mistake order
        # (so that label index 0 => mistake #1, etc.)
        values = [mistake_counts.get(mk, 0) for mk in labels]
        values += [values[0]]  # close the polygon

        # Plot the line for this circle
        ax.plot(angles, values, 'o-', linewidth=2, label=circle_name, color=color)
        ax.fill(angles, values, alpha=0.25, color=color)

    # Set up the axis ticks and labels
    ax.set_thetagrids(np.degrees(angles[:-1]), [f"Dominated Strategy {k}" for k in labels])
    ax.set_title(f"Dominated Strategies Count for {model_name} by Circle", y=1.1)
    ax.grid(True)

  
    plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))
    plt.show()

'''
- Mistake 1: Making an offer worse than your previous offer. This occurs when you reject an offer better for you than the one you subsequently propose. 
- Mistake 2: Making an offer worse for you than your outside offer. This happens if you propose giving away so much that what you keep is worth less than your guaranteed alternative, which is your outside offer.
- Mistake 3: Offering no items or all items. Offering nothing (or everything) to the opponent (in the early or middle rounds) can be a clear suboptimal move. 
- Mistake 4: Accepting an offer worse for you than your outside offer. This occurs if you accept a division that yields a payoff lower than your guaranteed fallback.
- Mistake 5: Walking away from an offer better than your outside offer. This occurs when you reject a division that actually yields a higher payoff than your fallback.
'''
circles = [0, 1, 2, 3, 4, 5, 6]



ds_data_4o = {}
ds_data_gemini = {}
#/Users/gabesmithline/Desktop/caif_negotiation/experiments/gemini_2.0_final/gemini_2.0_1_28_2025_circle_0.json
for circle in circles:
    print(f"Analyzing circle {circle}")
    print("="*100)
    json_FILE_PATH = f"/Users/gabesmithline/Desktop/caif_negotiation/experiments/4o_final/4o_1_28_2025_100_circle_{circle}.json"
    #json_FILE_PATH = f"/Users/gabesmithline/Desktop/caif_negotiation/experiments/gemini_2.0_final/gemini_2.0_1_28_2025_circle_{circle}.json"
    mistake_counts = analyze_circle6(json_FILE_PATH)
    print(mistake_counts)
    ds_data_gemini[f"4o_circle_{circle}"] = mistake_counts
#json_FILE_PATH = "/Users/gabesmithline/Desktop/caif_negotiation/experiments/test_mistakes_5games.json"





plot_mistakes_radar_multiple_circles(ds_data_gemini, "4o")

## Game Statistics: